In [155]:
""""""
import itertools
import logging
from enum import Enum
from pathlib import Path

import neurom as nm
import numpy as np
import pandas as pd
from lxml import etree
from neurom import NeuriteType
from scipy import stats

L = logging.getLogger(__name__)
pd.options.display.width = 0


class DISCRETE_FEATURE_NAMES(Enum):
    LEN = 'total_length'
    SURFACE_AREA = 'total_area_per_neurite'
    VOLUMES = 'neurite_volumes'
    NUMBER_OF_SECTIONS = 'number_of_sections'
    NUMBER_OF_BIFURCATIONS = 'number_of_bifurcations'
    NUMBER_OF_TERMINATIONS = 'number_of_terminations'


class CONTINUOUS_FEATURE_NAMES(Enum):
    SECTION_LEN = 'section_lengths'
    SECTION_RADIAL_DISTANCES = 'section_radial_distances'
    SECTION_PATH_DISTANCES = 'section_path_distances'
    PARTITION_ASYMMETRY = 'partition_asymmetry'
    SEGMENT_RADII = 'segment_radii'


NEURITES = (NeuriteType.soma,
            NeuriteType.axon,
            NeuriteType.basal_dendrite,
            NeuriteType.apical_dendrite,
            NeuriteType.undefined,)
NEURITE_NAMES = [type.name for type in NEURITES]

CUSTOM_FEATURE_LOAD = {
    DISCRETE_FEATURE_NAMES.SURFACE_AREA.value: {
        NeuriteType.soma.name: lambda neuron: nm.get('soma_surface_areas', neuron),
    }
}

MORPH_FILETYPES = ['.h5', '.swc', '.asc']


class Features(object):
    class INDEX(Enum):
        MTYPE = 'mtype'
        FILENAME = 'filename'
        NEURITE = 'neurite'

    _INDEX_NAMES = [index.value for index in INDEX]

    def __init__(self, index, discrete, continuous):
        self.discrete = pd.concat(discrete, keys=index, names=self._INDEX_NAMES)
        self.continuous = pd.concat(continuous, keys=index, names=self._INDEX_NAMES)


def get_discrete_features(neuron) -> pd.DataFrame:
    feature_names = [name.value for name in DISCRETE_FEATURE_NAMES]
    df = get_features(neuron, feature_names)
    df = df.applymap(np.sum)
    df.loc[NeuriteType.all.name] = df.sum()
    return df


def get_continuous_features(neuron) -> pd.DataFrame:
    feature_names = [name.value for name in CONTINUOUS_FEATURE_NAMES]
    df = get_features(neuron, feature_names)
    # `np.concatenate(x).tolist()` is used instead `np.concatenate(x)` to treat the return object
    # as a single value. Without it pandas treat it as a Series and tries to broadcast it.
    # When `np.concatenate(x).tolist()` returns a list of length equal to len(df) => may be an error
    df.loc[NeuriteType.all.name] = df.aggregate(lambda x: np.concatenate(x).tolist())
    return df


def get_features(neuron, feature_names) -> pd.DataFrame:
    df = pd.DataFrame(index=NEURITE_NAMES, columns=feature_names)
    for neurite, feature_name in itertools.product(NEURITES, feature_names):
        val = None
        if feature_name in CUSTOM_FEATURE_LOAD:
            if neurite.name in CUSTOM_FEATURE_LOAD[feature_name]:
                val = CUSTOM_FEATURE_LOAD[feature_name][neurite.name](neuron)
        if val is None:
            val = nm.get(feature_name, neuron, neurite_type=neurite)
        df.loc[neurite.name, feature_name] = val.tolist()
    return df


def get_mtype_dict(db_file: Path) -> dict:
    root = etree.parse(str(db_file)).getroot()
    mtype_dict = {}
    for morphology in root.iterfind('.//morphology'):
        name = morphology.findtext('name')
        if not name:
            L.warning('Empty morphology name in %s', db_file)
        mtype = morphology.findtext('mtype')
        if not mtype:
            L.warning('Empty morphology mtype in %s', db_file)
        if name in mtype_dict and mtype_dict[name] != mtype:
            L.warning('Multiple mtypes %s %s for %s', mtype, mtype_dict[name], name)
        mtype_dict[name] = mtype
    return mtype_dict


def get_valid_morph_features(morph_dirpath: Path) -> Features:
    if not morph_dirpath.is_dir():
        raise ValueError(
            '"{}" must be a directory with morphology files'.format(morph_dirpath))
    mtype_dict = get_mtype_dict(morph_dirpath.joinpath('neuronDB.xml'))
    index, discrete, continuous = [], [], []
    for file in morph_dirpath.iterdir():
        if file.suffix in MORPH_FILETYPES:
            neuron = nm.load_neuron(str(file))
            mtype = mtype_dict[neuron.name]
            index.append((mtype, neuron.name))
            discrete.append(get_discrete_features(neuron))
            continuous.append(get_continuous_features(neuron))
    return Features(index, discrete, continuous)


def get_test_morph_features(morph_dirpath: Path) -> Features:
    if not morph_dirpath.is_dir():
        raise ValueError(
            '"{}" must be a directory'.format(morph_dirpath))
    index, discrete, continuous = [], [], []
    for mtype_dir in morph_dirpath.iterdir():
        mtype = mtype_dir.name
        for file in mtype_dir.iterdir():
            if file.suffix in MORPH_FILETYPES:
                neuron = nm.load_neuron(str(file))
                index.append((mtype, neuron.name))
                discrete.append(get_discrete_features(neuron))
                continuous.append(get_continuous_features(neuron))
    return Features(index, discrete, continuous)


def ks_2samp(a, b):
    return stats.ks_2samp(a, b) + (len(a),)


def get_ks_of_valid_features(valid_features):
    def ks_valid(feature_series):
        def ks(a, b):
            b = np.concatenate(b)
            if a and b.size:
                return ks_2samp(a, b)

        fs_list = feature_series.to_list()
        return [ks(fs_list[i], fs_list[:i] + fs_list[i + 1:]) for i in range(0, len(fs_list))]

    return valid_features.continuous \
        .groupby([Features.INDEX.MTYPE.value, Features.INDEX.NEURITE.value]) \
        .transform(ks_valid)


def get_ks_of_test_features(test_features, valid_features):
    def ks_test(file_series):
        mtype = file_series.index.get_level_values('mtype').unique()[0]
        if not mtype in neurite_feature_distr.index.levels[0]:
            return None
        mtype_series = neurite_feature_distr.loc[mtype][file_series.name]
        return [ks_2samp(fm[0], fm[1])
                if fm[0] and fm[1] else None for fm in zip(file_series, mtype_series)]

    neurite_feature_distr = valid_features.continuous \
        .groupby([Features.INDEX.MTYPE.value, Features.INDEX.NEURITE.value]) \
        .agg(lambda feature: np.concatenate(feature).tolist())

    return test_features.continuous \
        .groupby([Features.INDEX.MTYPE.value, Features.INDEX.FILENAME.value]).transform(ks_test)


def discrete_z_score(valid_features: Features, test_features: Features):
    valid_mean = valid_features.discrete.groupby(
        [Features.INDEX.MTYPE.value, Features.INDEX.NEURITE.value]).mean()
    valid_std = valid_features.discrete.groupby(
        [Features.INDEX.MTYPE.value, Features.INDEX.NEURITE.value]).std()
    return ((test_features.discrete - valid_mean) / valid_std).dropna(how='all')


def discrete_report(z_score: pd.DataFrame, p_value=0.05):
    assert 0. <= p_value <= 1.
    threshold = np.abs(stats.norm.ppf(p_value / 2.))
    # some cells in z_score are NaN so we use `failed_neurites` + `any`
    # instead of `valid_neurites` + `all`.
    failed_neurites = (z_score.abs() > threshold).any(axis=1)
    return (~failed_neurites).groupby(['filename', 'mtype']).all()


In [71]:
# valid_features = build_valid_morphologies(Path('../tests/data/morphologies/valid/all'))
# import shutil
# L23_BTC
# filenames = set(valid_features.discrete.loc['L5_TPC'].index.get_level_values('filename'))
# filenames
# for filename in filenames:
#     shutil.copy2(
#         '/home/sanin/workspace/morph-validator/tests/data/morphologies/valid/all/' + filename + '.h5',
#         '/home/sanin/workspace/morph-validator/tests/data/morphologies/valid/mini')

In [156]:
valid_features = build_valid_morphologies(Path('../tests/data/morphologies/valid/mini'))

In [157]:
test_features = build_test_morphologies(Path('../tests/data/morphologies/test'))

In [158]:
z_score = discrete_z_score(valid_features, test_features)
discrete_report(z_score)


filename        mtype  
C040426         L5_MC      True
C050896A-I      L5_MC      True
C290500C-I4     L5_MC      True
mtC031100A_idB  L23_BTC    True
dtype: bool

In [161]:
valid_ks = get_ks_of_valid_features(valid_features)
valid_ks

section_lengths  \
mtype   filename                  neurite                                                              
L23_BTC C210401C                  soma                                                          None   
                                  axon              (0.19442848965980639, 4.22459922599927e-09, 281)   
                                  basal_dendrite      (0.14330143540669857, 0.22719573169336205, 55)   
                                  apical_dendrite                                               None   
                                  undefined                                                     None   
...                                                                                              ...   
L5_TPC  rat_20170523_E1_LH2_cell1 axon               (0.2559411711954085, 9.89319737243477e-13, 245)   
                                  basal_dendrite       (0.09636650868878358, 0.5597696267161392, 90)   
                                  apical_dendrite     (0.15895610913404506, 0.04322940735041114, 99)   
                                  undefined                                                     None   
                                  all              (0.22594127463006553, 3.9968028886505635e-15, ...   

                                                                            section_radial_distances  \
mtype   filename                  neurite                                                              
L23_BTC C210401C                  soma                                                          None   
                                  axon             (0.47410397431749746, 2.738166969868195e-51, 281)   
                                  basal_dendrite       (0.09614234449760765, 0.7011390763718283, 55)   
                                  apical_dendrite                                               None   
                                  undefined                                                     None   
...                                                                                              ...   
L5_TPC  rat_20170523_E1_LH2_cell1 axon                (0.1038676864342724, 0.01877043125658573, 245)   
                                  basal_dendrite      (0.18304370721432334, 0.02484402781049544, 90)   
                                  apical_dendrite     (0.17071066537258708, 0.02419461006903345, 99)   
                                  undefined                                                     None   
                                  all              (0.09714219407883831, 0.0020929689331080636, 434)   

                                                                              section_path_distances  \
mtype   filename                  neurite                                                              
L23_BTC C210401C                  soma                                                          None   
                                  axon             (0.43307667421546425, 7.238951991722821e-43, 281)   
                                  basal_dendrite       (0.09973086124401914, 0.6577285330497541, 55)   
                                  apical_dendrite                                               None   
                                  undefined                                                     None   
...                                                                                              ...   
L5_TPC  rat_20170523_E1_LH2_cell1 axon               (0.1189335998052705, 0.004432607154251644, 245)   
                                  basal_dendrite        (0.1291205897840969, 0.2193805393354571, 90)   
                                  apical_dendrite      (0.08044861425644344, 0.6930934989347561, 99)   
                                  undefined                                                     None   
                                  all              (0.12965664334891533, 9.972688787107664e-06, 434)   

                          

In [162]:
test_ks = get_ks_of_test_features(test_features, valid_features)
test_ks

section_lengths  \
mtype   filename       neurite                                                              
L5_MC   C040426        soma                                                          None   
                       axon              (0.1391161763750893, 0.0003542457426518597, 232)   
                       basal_dendrite      (0.24360859266519644, 0.03865565996219866, 33)   
                       apical_dendrite                                               None   
                       undefined                                                     None   
                       all              (0.13981548001663704, 9.371494500731714e-05, 265)   
        C050896A-I     soma                                                          None   
                       axon              (0.1119301375651439, 0.0004350034932669722, 361)   
                       basal_dendrite        (0.2169811320754717, 0.0070074163070728, 63)   
                       apical_dendrite                                               None   
                       undefined                                                     None   
                       all              (0.10897077173214353, 0.00016932483195619685, ...   
        C290500C-I4    soma                                                          None   
                       axon             (0.13131495728967818, 0.00013861030867812207, ...   
                       basal_dendrite      (0.18859394725169742, 0.01062991821665793, 79)   
                       apical_dendrite                                               None   
                       undefined                                                     None   
                       all               (0.08058805783610232, 0.020984740765956467, 372)   
Unknown ca3b-N2.CNG    soma                                                          None   
                       axon                                                          None   
                       basal_dendrite                                                None   
                       apical_dendrite                                               None   
                       undefined                                                     None   
                       all                                                           None   
L23_BTC mtC031100A_idB soma                                                          None   
                       axon             (0.14393153526970953, 2.6174006673818084e-05, ...   
                       basal_dendrite      (0.16461969403145874, 0.07821517246657317, 63)   
                       apical_dendrite                                               None   
                       undefined                                                     None   
                       all              (0.11590694308109666, 0.0002863598345367313, 351)   

                                                                 section_radial_distances  \
mtype   filename       neurite                                                              
L5_MC   C040426        soma                                                          None   
                       axon             (0.16988927852457952, 5.080399893553356e-06, 232)   
                       basal_dendrite    (0.30727762803234504, 0.0038183191843642605, 33)   
                       apical_dendrite                                               None   
                       undefined                                                     None   
                       all              (0.13848073505501568, 0.0001132425319960717, 265)   
        C050896A-I     soma                                                          None   
                       axon             (0.3124096853790736, 1.2433381212604616e-28, 361)   
                       basal_dendrite      (0.16636717580113808, 0.07104325192253647, 63)   
                       apical_dendrite                                    

In [165]:
valid_ks.groupby(['mtype', 'neurite']).apply(lambda x: print(x, '@@@@@@@@'))

                                 section_lengths section_radial_distances  \
mtype   filename         neurite                                            
L23_BTC C210401C         soma               None                     None   
        mtC020502A_idA   soma               None                     None   
        mtC031100A_idB   soma               None                     None   
        C200199B-I4      soma               None                     None   
        C190898A-I1      soma               None                     None   
        mtC061100A_idC   soma               None                     None   
        C220797A-I2      soma               None                     None   
        mtC121100B_idJ   soma               None                     None   
        mtC240300A_idB   soma               None                     None   
        mtC020200_14_idA soma               None                     None   

                                 section_path_distances partition_asymmetry

""
